## Importing Required Python Libraries

This section is dedicated to importing all the necessary Python libraries that are essential for executing the subsequent parts of this notebook. 

In [1]:
# Importing Required Python Libraries
import yfinance as yf
import pandas as pd
from sklearn.impute import SimpleImputer, KNNImputer
import numpy as np
from datetime import datetime
import os

## Phase 1: Data Collection

In this initial phase, we gather historical stock data for a specified list of ticker symbols within a defined date range. Utilizing the **Collect_Stock_Data** function, we iterate through each ticker symbol, retrieving its historical market data using the **yfinance** library. Subsequently, the data is stored in a dictionary after ensuring its validity and sufficiency. Feedback messages are provided to indicate the success or failure of data collection for each ticker symbol.


In [2]:
def Collect_Stock_Data(start_date, end_date, list_of_ticker_symbols):

    """
    Collects historical stock data for a given list of ticker symbols within a specified date range.

    This function iterates through each ticker symbol, retrieves its historical market data using
    the yfinance library, and stores the data in a dictionary if it meets certain conditions. It checks
    whether the data is not empty and if the number of rows meets at least 80% of the expected number of
    days within the given timeframe, assuming 250 trading days in a year.

    Parameters:
    - start_date (str): The start date for the data collection in "YYYY-MM-DD" format.
    - end_date (str): The end date for the data collection in "YYYY-MM-DD" format.
    - list_of_ticker_symbols (list): A list of string ticker symbols for which to collect data.

    Returns:
    - dict: A dictionary with ticker symbols as keys and their corresponding historical data DataFrames as values.

    Note:
    The function prints messages indicating the success of data collection for each ticker or the failure
    to meet the data requirements.
    """
    
    # Dictionary to store the historical data DataFrames
    dataframes= {}
    
    # Extracting Stock Data
    for stock in list_of_ticker_symbols:        

        stk = yf.Ticker(stock)
        
        current_data = stk.history(start=start_date, end=end_date)
        
        # Calculate the number of rows and check against the 80% threshold of expected days
        number_rows = current_data.shape[0]
        expected_days = (250/365)*((datetime.strptime(end_date, "%Y-%m-%d") - datetime.strptime(start_date, "%Y-%m-%d")).days)*0.8
        
        if (current_data.empty==False) and (number_rows>=expected_days):
            print(stock, 'data has been collected successfully.')
            # Store the DataFrame
            dataframes[stock] = current_data
        else:
            print('Sorry we can not do prediction for {}, Since No data found for this stock in this time frame.'.format(stock))
            
    return dataframes


In [3]:
list_of_ticker_symbols = ['MSFT', 'UPS']
start_date = '2013-11-10'
end_date = '2023-11-10'

In [4]:
dataframes = Collect_Stock_Data(start_date, end_date, list_of_ticker_symbols)

MSFT data has been collected successfully.
UPS data has been collected successfully.


## Phase 2: Data Cleaning - Handling Missing Values

In this phase, continuous NaN (Not a Number) sequences within the dataset are addressed through data cleaning. Rows that are part of a continuous sequence of NaNs exceeding a predefined threshold are identified and removed by the Drop_Continuous_NaNs function. The balance between data retention and the elimination of potentially problematic missing data sequences is achieved by specifying the maximum allowed sequence length of NaNs.

In [5]:
dataframes['MSFT']

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2013-11-11 00:00:00-05:00,31.440089,31.515165,31.156469,31.356672,26872500,0.0,0.0
2013-11-12 00:00:00-05:00,31.181494,31.365011,31.031342,31.164810,31651600,0.0,0.0
2013-11-13 00:00:00-05:00,30.847814,31.832142,30.781082,31.832142,44957600,0.0,0.0
2013-11-14 00:00:00-05:00,31.590239,31.807126,31.465114,31.715366,46183700,0.0,0.0
2013-11-15 00:00:00-05:00,31.656982,31.715374,31.465122,31.565222,50601300,0.0,0.0
...,...,...,...,...,...,...,...
2023-11-03 00:00:00-04:00,348.277763,353.019363,345.986641,351.435486,23624000,0.0,0.0
2023-11-06 00:00:00-05:00,352.083018,356.157196,351.983399,355.151093,23828300,0.0,0.0
2023-11-07 00:00:00-05:00,358.009986,361.058148,356.246842,359.135620,25833900,0.0,0.0


In [6]:
def Drop_Continuous_NaNs(df, max_allowed_continuous_nans=3):
    
    """
    Filters out rows from a pandas DataFrame that are part of a continuous sequence of NaNs (Not a Number)
    exceeding a specified threshold in any column.

    The function identifies sequences of continuous NaNs across all columns and drops any rows that are
    part of a sequence longer than `max_allowed_continuous_nans`. It handles each column independently,
    ensuring that no column has continuous NaNs exceeding the threshold.

    Parameters:
    - df (pandas.DataFrame): The DataFrame to be processed.
    - max_allowed_continuous_nans (int, optional): The maximum allowed length of continuous NaN sequences.
      Rows that are part of a sequence longer than this will be dropped. Defaults to 3.

    Returns:
    - pandas.DataFrame: A DataFrame with rows in continuous NaN sequences beyond the allowed threshold removed.

    Note:
    The function directly analyzes the NaN presence in each row, groups continuous NaN sequences,
    and filters based on the specified threshold. It preserves the original DataFrame structure
    and data integrity, except for the removal of specified sequences.
    """

    # Identify rows that are completely NaN
    all_nan_rows = df.isna().all(axis=1)
    
    # Group consecutive NaN rows to identify continuous sequences
    all_nan_groups = (~all_nan_rows).cumsum()
    
    # Calculate the size of each group of continuous NaN rows
    group_sizes = all_nan_groups.map(all_nan_groups.value_counts())
    
    # Generate a mask for valid rows: either not part of a NaN sequence or within the allowed NaN sequence length
    valid_rows_mask = (group_sizes <= max_allowed_continuous_nans) | (~all_nan_rows)
    
    # Apply the mask to filter the DataFrame, removing unwanted continuous NaN sequences
    df_filtered = df[valid_rows_mask]
    
    return df_filtered

## Phase 3: Data Preprocessing - NaN Values Cleaning

In this phase, NaN values across multiple pandas DataFrames are cleaned to ensure data quality for analysis. The Clean_NaN_Values function is employed to methodically process each DataFrame contained within a provided dictionary. Initially, rows featuring continuous sequences of NaN values exceeding a predefined threshold are identified and removed. Subsequently, any remaining NaNs within specific columns are addressed through deletion, filling, or imputation, contingent upon the nature of the data and the proportion of missing valuesis..

In [7]:
def Clean_NaN_Values(dataframes):
    
    """
    Cleans NaN values across a collection of pandas DataFrames by first removing rows with continuous
    NaNs beyond a set threshold, then addressing remaining NaNs in specific columns through either deletion,
    filling, or imputation based on the nature of the data and the proportion of missing values.

    Parameters:
    - dataframes (dict): A dictionary where keys are identifiers (e.g., ticker symbols) and values are
      pandas DataFrames containing stock market data.

    Returns:
    - dict: The same dictionary with DataFrames cleaned of NaN values according to the specified rules.

    Note:
    This function specifically treats financial data columns differently based on their importance and
    the feasibility of imputation. It also provides warnings for DataFrames where critical columns like
    'Close' have a high percentage of NaN values, potentially indicating inadequate data for reliable analysis.
    """
    
    for ticker in dataframes:
        df = dataframes[ticker]

        # Drop rows with continuous NaNs longer than the threshold
        df = Drop_Continuous_NaNs(df, max_allowed_continuous_nans=3)    

        # Calculate the percentage of NaNs in the 'Close' column
        closing_price_NaN = df['Close'].isna().mean() * 100
        NaN_threshold_for_close = 12

        # Check if 'Close' column NaN percentage exceeds threshold
        if closing_price_NaN > NaN_threshold_for_close:
            print('Sorry we can not predict {} since more than {} of the Close Price column is NaN'.format(ticker, NaN_threshold_for_close))
            
        else:
            for column in df.columns:
                # Fill NaNs with 0 for 'Dividends' and 'Stock Splits'
                if column in ['Dividends', 'Stock Splits']:
                    df[column].fillna(0, inplace=True)

                # Analyze and handle NaNs for financial columns differently
                if column in ['Open', 'High', 'Low', 'Close', 'Volume']:
                    # NaN analysis on the cleaned DataFrame
                    nan_percent = df[column].isna().mean() * 100

                    # If >50% NaNs, drop the column with a warning
                    if nan_percent > 50:
                        
                            # Attempt to acquire more complete data or use domain knowledge
                            print(f"Column: {column} has >50% NaNs. Dropping this column. Consider acquiring more complete data.")
                            del df[column]
                    # For NaN percentages between 0 and 50
                    elif 0 < nan_percent <= 50:
                        # Use forward fill and backward fill for low NaN percentage
                        if nan_percent < 5: 
                            # Forward fill for financial time series data
                            df[column]= df[column].fillna(method='ffill').fillna(method='bfill')
                            
                        else:
                            # Apply KNN imputation for higher NaN percentages to the entire DataFrame (considering all columns)                            
                            columns_for_imputation = df[['Open', 'High', 'Low', 'Close', 'Volume']]
                            knn_imputer = KNNImputer(n_neighbors=5)
                            imputed_data = knn_imputer.fit_transform(columns_for_imputation)
                            imputed_df = pd.DataFrame(imputed_data, columns=columns_for_imputation.columns)
                            
                            # Merge the imputed 'Volume' column back into the original DataFrame (Step 5)
                            df[column] = imputed_df[column]
                    
                    else:
                        # No action needed for columns without NaN values
                        pass
        # Update the DataFrame in the dictionary
        dataframes[ticker] = df

    return dataframes

In [8]:
dataframes = Clean_NaN_Values(dataframes)

## Phase 4: Data Enhancement - Adjusting Stock Prices

This stage is dedicated to refining the dataset by calculating adjusted closing prices for stocks, encapsulating the effect of corporate actions like stock splits and dividends. The Calculate_Adjusted_Price function updates the 'Adjusted_Close' column within each DataFrame from a collection keyed by ticker symbols. Adjustments ensure that the historical prices reflect true stock value post-corporate events, providing a more accurate basis for financial analysis.

Adjustments are applied in two folds: prices are recalibrated for stock splits by direct modification, and for dividends, by adjusting the price to mirror the payout’s impact on stock value.

In [9]:
def Calculate_Adjusted_Price(dataframes):

    """
    Calculates the adjusted closing price for stocks within each DataFrame in a collection, taking into account
    stock splits and dividends. The adjusted closing price reflects the stock's value after accounting for
    corporate actions.

    Adjustments are made by modifying the 'Adjusted_Close' column based on the 'Stock Splits' and 'Dividends'
    columns. Stock splits adjust the price directly, while dividends adjust the price to reflect the payout's
    impact on the stock's value.

    Parameters:
    - dataframes (dict): A dictionary of pandas DataFrames keyed by ticker symbols 

    Returns:
    - dict: The same dictionary with each DataFrame's 'Adjusted_Close' column updated to reflect the adjusted
      closing prices.

    Note:
    The function processes each DataFrame independently, ensuring the date column is in the correct format
    and sorted in ascending order before making adjustments.
    """
    
    for ticker in dataframes:
        df = dataframes[ticker]
        
        # Ensure the 'Date' column is in 'YYYY-MM-DD' format and set as datetime type
        df.reset_index(inplace= True)
        df['Date'] = df['Date'].apply(lambda x: str(x).split(' ')[0])
        df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
            
        df.sort_values('Date', ascending=True, inplace=True)
        
        # Initialize 'Adjusted_Close' column with existing close prices
        df['Adjusted_Close'] = df['Close']
        
        # Adjust 'Adjusted_Close' for stock splits
        for i in df[df['Stock Splits'] != 0].index:
            split_factor = df.at[i, 'Stock Splits']
            df.loc[:i-1, 'Adjusted_Close'] /= split_factor
        
        # Adjust 'Adjusted_Close' for dividends
        for i in df[df['Dividends'] != 0].index:
            dividend = df.at[i, 'Dividends']
            price = df.at[i-1, 'Close']
            Dividend_Multiplier = 1 - (dividend/price)
            df.loc[:i-1, 'Adjusted_Close'] *= Dividend_Multiplier
            
        dataframes[ticker] = df
        
    return dataframes

In [10]:
dataframes = Calculate_Adjusted_Price(dataframes)

## Phase 5: Momentum Analysis - Calculating RSI

In this phase, the dataset undergoes enhancement through the calculation of the Relative Strength Index (RSI) for stocks, a pivotal momentum indicator. The calculate_rsi function is integral to this process, applying the RSI calculation across specified columns in each DataFrame within the collection.

This function, by utilizing a typical 14-day period for evaluation, enables an assessment of stock momentum by segregating price movements into gains and losses. These segregated movements are then averaged and utilized to compute the RSI, enriching each DataFrame with a new 'RSI' column.

In [11]:
def Calculate_RSI(dataframes, column='Adjusted_Close', period=14):
    
    """
    Calculates the Relative Strength Index (RSI) for the specified column in each DataFrame within a
    collection. RSI is a momentum indicator that measures the magnitude of recent price changes to evaluate
    overbought or oversold conditions in the price of a stock.

    The RSI is calculated using a specified period (typically 14 days) and is scaled from 0 to 100. An RSI
    above 70 is generally considered overbought, while an RSI below 30 is considered oversold.

    Parameters:
    - dataframes (dict): A dictionary of DataFrames keyed by ticker symbols
    - column (str, optional): The column name to calculate RSI for. Defaults to 'Adjusted_Close'.
    - period (int, optional): The period over which to calculate RSI, typically 14. Defaults to 14.

    Returns:
    - dict: The same dictionary of DataFrames, now including an 'RSI' column with the calculated RSI
      values for each stock.

    Note:
    This function iterates over each DataFrame, calculating the RSI based on price changes in the specified
    column. It handles positive and negative changes separately to compute average gains and losses, which
    are then used to derive the RSI values.
    """
    
    for ticker in dataframes:
        df = dataframes[ticker]

        # Calculate price difference from the previous day
        delta = df[column].diff(1)
        # Isolate gains and losses from the delta
        gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()

        # Calculate the Relative Strength (RS)
        rs = gain / loss
        df['RSI'] = 100 - (100 / (1 + rs))
        
        dataframes[ticker] = df

    return dataframes


In [12]:
dataframes = Calculate_RSI(dataframes, column='Adjusted_Close', period=14)

## Phase 6: Trend Analysis - Implementing MACD

In this phase, the dataset is enhanced with the inclusion of the MACD and its signal line, tools that are utilized to observe and predict potential shifts in stock price movements. The MACD is derived by examining the difference between two averages of stock prices over differing time spans, one short-term and the other long-term, while the signal line, an additional average, serves to validate the insights provided by the MACD.

The calculate_macd function is employed to perform these calculations for each stock within the dataset, resulting in the addition of two columns: 'MACD' and 'Signal_Line'. These enhancements facilitate a simpler understanding of when a stock’s price trend may be about to change, offering a basis for more informed buying or selling decisions.

In [13]:
def Calculate_MACD(dataframes, column='Adjusted_Close', short_period=12, long_period=26, signal_period=9):
    
    """
    Calculates the Moving Average Convergence Divergence (MACD) and its signal line for the specified
    column in each DataFrame within a collection. MACD is a trend-following momentum indicator
    that demonstrates the relationship between two moving averages of a stock's price. The MACD is
    calculated by subtracting the long-term Exponential Moving Average (EMA) from the short-term EMA,
    with the signal line being an EMA of the MACD itself.

    Parameters:
    - dataframes (dict): A dictionary of DataFrames keyed by ticker symbols).
    - column (str, optional): The column name to calculate MACD for. Defaults to 'Adjusted_Close'.
    - short_period (int, optional): The period for the short-term EMA. Defaults to 12.
    - long_period (int, optional): The period for the long-term EMA. Defaults to 26.
    - signal_period (int, optional): The period for the signal line EMA. Defaults to 9.

    Returns:
    - dict: The same dictionary of DataFrames, now including 'MACD' and 'Signal_Line' columns with
      the calculated values.

    Note:
    This function adds two columns to each DataFrame: 'MACD', which is the difference between the short-term
    and long-term EMAs of the specified column, and 'Signal_Line', which is the EMA of the 'MACD' column over
    the specified signal period. These indicators are commonly used to identify potential trend changes and
    trading opportunities.
    """

    for ticker in dataframes:
        df = dataframes[ticker]
        
        # Calculate the short-term and long-term EMAs
        short_ema = df[column].ewm(span=short_period, adjust=False).mean()
        long_ema = df[column].ewm(span=long_period, adjust=False).mean()
        
        # Compute the MACD by subtracting the long-term EMA from the short-term EMA
        df['MACD'] = short_ema - long_ema
        
        # Calculate the signal line, which is the EMA of the MACD
        df['Signal_Line'] = df['MACD'].ewm(span=signal_period, adjust=False).mean()

        dataframes[ticker] = df

    return dataframes

In [14]:
dataframes = Calculate_MACD(dataframes, column='Adjusted_Close', short_period=12, long_period=26, signal_period=9)

## Phase 7: Enhancing Analysis with Bollinger Bands

In this phase, the dataset is further augmented through the calculation of Bollinger Bands for each stock, employing a standard method that uses a specified moving average period (often 20 days) and a set number of standard deviations (typically 2) from this average to define upper and lower bands around the stock's price movements. The inclusion of these bands in the dataset, represented by 'Upper_Band' and 'Lower_Band' columns, serves as a mechanism to gauge market volatility and the relative positioning of stock prices.

In [15]:
def Calculate_Bollinger_Bands(dataframes, column='Adjusted_Close', period=20):
    
    """
    Calculates Bollinger Bands for the specified column in each DataFrame within a collection.
    Bollinger Bands are a volatility indicator and consist of three lines: the middle band is an N-period
    moving average (MA), the upper band is K standard deviations above the MA, and the lower band is K
    standard deviations below the MA. This function sets N to the specified period (commonly 20) and K
    to 2, following the standard practice.

    Parameters:
    - dataframes (dict): A dictionary of DataFrames keyed by ticker symbols.
    - column (str, optional): The column name to calculate Bollinger Bands for. Defaults to 'Adjusted_Close'.
    - period (int, optional): The period over which the moving average and standard deviation are calculated.
      Defaults to 20.

    Returns:
    - dict: The same dictionary of DataFrames, now including 'Upper_Band' and 'Lower_Band' columns with
      the calculated Bollinger Bands.

    Note:
    This function enhances each DataFrame with two new columns: 'Upper_Band' and 'Lower_Band', representing
    the calculated upper and lower Bollinger Bands, respectively. These bands are used to assess whether
    prices are high or low on a relative basis and can indicate potential market volatility or price
    consolidations.
    """

    for ticker in dataframes:
        df = dataframes[ticker]
        
        # Calculate the moving average (MA) and standard deviation (STD) for the specified period
        MA = df[column].rolling(window=period).mean()
        STD = df[column].rolling(window=period).std()

        # Compute the upper and lower Bollinger Bands
        df['Upper_Band'] = MA + (STD * 2)
        df['Lower_Band'] = MA - (STD * 2)

        dataframes[ticker] = df

    return dataframes


In [16]:
dataframes = Calculate_Bollinger_Bands(dataframes, column='Adjusted_Close', period=20)

## Phase 8: Expanding the Dataset with Feature Engineering

In this phase, the dataset undergoes an enhancement through the feature engineering. Engineer_Features function enriches each DataFrame with new variables that capture both the momentum and the temporal dynamics of stock prices.

This augmentation includes the computation of short-term and medium-term moving averages to pinpoint trends, the calculation of On-Balance Volume (OBV) to gauge buying and selling pressure, and the projection of future price targets across multiple time horizons to assist in predictive modeling. Moreover, the incorporation of time-based features such as the day of the week and month embeds temporal patterns into the analysis, offering insights into periodic fluctuations in stock activity.

In [17]:
def Engineer_Features(dataframes):

    """
    Enriches each DataFrame with additional features useful for financial analysis and machine learning models.
    Features include moving averages, On-Balance Volume (OBV), future price targets for various horizons,
    and time-based features like day of the week and month.

    Parameters:
    - dataframes (dict): A dictionary of DataFrames keyed by ticker symbols.

    Returns:
    - dict: The same dictionary of pandas DataFrames, now augmented with additional features including
      moving averages (MA10, MA20), OBV, future targets (Target_1, Target_2, Target_7, Target_14, Target_28),
      and time-based features (Day_of_Week, Month).

    Note:
    The function aims to facilitate the identification of trends, prediction of future prices, and
    capturing of temporal patterns in stock data.
    """
    
    for ticker in dataframes:
        df = dataframes[ticker]

        # Calculate 10-day and 20-day moving averages
        df['MA10'] = df['Adjusted_Close'].rolling(window=10).mean()
        df['MA20'] = df['Adjusted_Close'].rolling(window=20).mean()
        
        # Compute On-Balance Volume (OBV)
        df['OBV'] = (np.sign(df['Adjusted_Close'].diff()) * df['Volume']).fillna(0).cumsum()

        # Define target variables for various prediction horizons
        df['Target_1'] = df['Adjusted_Close'].shift(-1)
        df['Target_2'] = df['Adjusted_Close'].shift(-2)
        df['Target_7'] = df['Adjusted_Close'].shift(-7)
        df['Target_14'] = df['Adjusted_Close'].shift(-14)
        df['Target_28'] = df['Adjusted_Close'].shift(-28)

        # Extract day of the week and month from the 'Date' column for time-based features
        df['Day_of_Week'] = df['Date'].dt.dayofweek
        df['Month'] = df['Date'].dt.month
        
        dataframes[ticker] = df

    return dataframes


In [18]:
dataframes = Engineer_Features(dataframes)

## Phase 9: Consolidating Market Indices Data

In this phase, the focus shifts towards assembling a comprehensive view of the market by fetching, cleaning, and processing historical data for a predefined set of major stock indices. The Use_Market_Data function stands at the core of this process, tasked with retrieving data over a specific date range, ensuring its quality by filtering based on the allowed percentage of NaN values in the 'Close' column, and calculating adjusted closing prices that account for corporate actions like stock splits and dividends.

The function operates on a dictionary of stock indices, fetching data for each through the yfinance library, and then rigorously checks for data completeness and the extent of missing values. Indices failing to meet the set criteria are excluded to maintain the integrity of the analysis. For those passing the checks, the function proceeds to adjust closing prices, merge the data into a unified DataFrame, and fill any gaps, thus crafting a dataset that reflects a holistic and accurate picture of market behavior.

In [19]:
def Use_Market_Data(dataframes, start_date, end_date, NaN_threshold_for_close=10):

    """
    Retrieves, cleans, and processes historical market data for a set of predefined stock indices
    over a specified date range. The function filters the data based on a threshold for the percentage
    of NaN values allowed in the 'Close' column and the expected data completeness. It calculates adjusted
    closing prices for each index, merges the data into a single DataFrame, and fills any missing values.

    Parameters:
    - dataframes (dict): A dictionary to store the adjusted price DataFrames, keyed by index symbols.
    - start_date (str): The start date for the data retrieval in "YYYY-MM-DD" format.
    - end_date (str): The end date for the data retrieval in "YYYY-MM-DD" format.
    - NaN_threshold_for_close (int, optional): The maximum percentage of NaN values allowed in the 'Close'
      column before an index is excluded. Defaults to 10%.

    Returns:
    - pd.DataFrame: A DataFrame containing the adjusted closing prices for each index, with dates aligned
      and missing values interpolated.

    Note:
    The function leverages the yfinance library to fetch the data, filters based on the specified criteria,
    and uses custom logic to adjust prices for stock splits and dividends.
    """

    # Predefined stock indices symbols
    stock_indices_dict = {'Dow Jones Industrial Average': 'DJIA',
                          'Nasdaq Composite': 'COMP',
                          'Russell 2000': 'RUT',
                          'MSCI World Index': 'MSCI',
                          'FTSE 100 (Financial Times Stock Exchange 100 Index)':'UKX',
                          'Nikkei 225': '^N225',
                          'Shanghai Composite Index': '000001.SS',
                          'DAX': 'DAX',
                          'SPY': 'SPY',
                          'VIX': 'VIX'}

    # Dictionary to hold the cleaned and processed index data
    df_indices = {}
    
    # Iterate over each index symbol to fetch and clean data
    for symbol in stock_indices_dict:
        index = stock_indices_dict[symbol]
        ind = yf.Ticker(index)
        # Fetch historical market data using yfinance
        current_data = ind.history(start=start_date, end=end_date)

        # Calculate the percentage of NaN values in 'Close' column
        closing_NaN = current_data['Close'].isna().mean() * 100
        # Calculate the actual number of rows fetched
        number_rows = current_data.shape[0]
        # Calculate the expected number of rows based on date range and trading days 
        expected_days = ((250/365)*(datetime.strptime(end_date, "%Y-%m-%d") - datetime.strptime(start_date, "%Y-%m-%d")).days)*0.8
        
        # Check if data meets completeness criteria and NaN threshold
        if (number_rows>= expected_days) and (closing_NaN < NaN_threshold_for_close):
            
            # Select relevant columns and fill NaN values
            current_data = current_data[['Dividends', 'Stock Splits', 'Close']]
            current_data['Dividends'].fillna(0, inplace=True)
            current_data['Stock Splits'].fillna(0, inplace=True)
            current_data['Close'] = current_data['Close'].interpolate(method='linear').fillna(method='ffill').fillna(method='bfill')
            df_indices[index] = current_data
        else:
            print('No data for ', index, '!')

    # DataFrame to aggregate adjusted market data
    Adjusted_Market = pd.DataFrame()
    if len(df_indices) !=0:
        # Adjust prices for stock splits and dividends
        df_indices = Calculate_Adjusted_Price(df_indices)

        # Merge adjusted prices from each index into a single DataFrame
        for ticker in df_indices:
            df = df_indices[ticker]
            df.rename(columns = {'Adjusted_Close': str('Adjusted_Close'+'_'+ticker)}, inplace=True)
            
            if Adjusted_Market.empty:
                Adjusted_Market = df[['Date', str('Adjusted_Close'+'_'+ticker)]]
            else:
                Adjusted_Market = pd.merge(Adjusted_Market, df[['Date', str('Adjusted_Close'+'_'+ticker)]], on='Date', how='outer', suffixes=('', ''))

    # Interpolate missing values to ensure completeness
    Adjusted_Market = Adjusted_Market.interpolate(method='linear').fillna(method='ffill').fillna(method='bfill')   
    
    return Adjusted_Market


In [20]:
Adjusted_Market = Use_Market_Data(dataframes, start_date, end_date, NaN_threshold_for_close=10)

No data for  DJIA !
No data for  COMP !
No data for  RUT !


C:\Users\enabi\AppData\Local\Temp\ipykernel_24808\2151122429.py:61: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  current_data['Close'] = current_data['Close'].interpolate(method='linear').fillna(method='ffill').fillna(method='bfill')
C:\Users\enabi\AppData\Local\Temp\ipykernel_24808\2151122429.py:61: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  current_data['Close'] = current_data['Close'].interpolate(method='linear').fillna(method='ffill').fillna(method='bfill')


No data for  UKX !


C:\Users\enabi\AppData\Local\Temp\ipykernel_24808\2151122429.py:61: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  current_data['Close'] = current_data['Close'].interpolate(method='linear').fillna(method='ffill').fillna(method='bfill')
C:\Users\enabi\AppData\Local\Temp\ipykernel_24808\2151122429.py:61: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  current_data['Close'] = current_data['Close'].interpolate(method='linear').fillna(method='ffill').fillna(method='bfill')


No data for  VIX !


C:\Users\enabi\AppData\Local\Temp\ipykernel_24808\2151122429.py:61: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  current_data['Close'] = current_data['Close'].interpolate(method='linear').fillna(method='ffill').fillna(method='bfill')
C:\Users\enabi\AppData\Local\Temp\ipykernel_24808\2151122429.py:83: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  Adjusted_Market = Adjusted_Market.interpolate(method='linear').fillna(method='ffill').fillna(method='bfill')


## Phase 10: Integrating Stock and Market Data

This phase involves the combination of individual stock data with aggregated market index data, a process executed by the Merge_stock_with_Market function. By aligning both datasets by date, each stock's performance is contextualized within the broader market environment, enhancing the depth and relevance of analyses. This merging not only allows for a direct comparison between a stock's movements and overall market trends but also enriches the stock dataset with a layer of market insight critical for comprehensive analysis.

Employing a 'left' join ensures the preservation of the original stock data structure while integrating adjusted market index information, thus maintaining the integrity of individual stock analyses.

In [21]:
def Merge_stock_with_Market(dataframes, Adjusted_Market):

    """
    Merges adjusted market index data with each individual stock DataFrame in a collection,
    aligning them by date. This operation enriches stock data with broader market context,
    facilitating analyses that require comparing stock performance against market indices.

    Parameters:
    - dataframes (dict): A dictionary of DataFrames keyed by ticker symbols.
    - Adjusted_Market (pd.DataFrame): A DataFrame containing adjusted market index data, with one column
      for dates and other columns for the adjusted closing prices of various market indices.

    Returns:
    - dict: The original dictionary of DataFrames, now with each stock DataFrame merged with the
      Adjusted_Market DataFrame on the 'Date' column.

    Note:
    The function ensures that the merge operation retains all rows from the stock DataFrames (using a
    'left' join).
    """

    # Iterate over each stock DataFrame to merge with market data
    for ticker in dataframes:
        df = dataframes[ticker]
        
        # Check if the Adjusted_Market DataFrame contains data
        if not Adjusted_Market.empty:
            # Merge the stock DataFrame with Adjusted_Market on 'Date', appending market data
            df = pd.merge(df, Adjusted_Market, on='Date', how='left', suffixes=('', ''))
            # Update the dictionary with the merged DataFrame
            dataframes[ticker] = df
        
    return dataframes

In [22]:
dataframes = Merge_stock_with_Market(dataframes, Adjusted_Market)

In [23]:
dataframes['MSFT']

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Adjusted_Close,RSI,...,Target_7,Target_14,Target_28,Day_of_Week,Month,Adjusted_Close_MSCI,Adjusted_Close_^N225,Adjusted_Close_000001.SS,Adjusted_Close_DAX,Adjusted_Close_SPY
0,2013-11-11,31.440089,31.515165,31.156469,31.356672,26872500,0.0,0.0,25.788608,NaN,...,25.865033,26.820667,25.669716,0,11,33.815454,14269.839844,2109.470947,16.194633,119.930829
1,2013-11-12,31.181494,31.365011,31.031342,31.164810,31651600,0.0,0.0,25.630815,NaN,...,26.088255,26.723015,25.544160,1,11,33.920666,14588.679688,2126.771973,16.194633,119.687370
2,2013-11-13,30.847814,31.832142,30.781082,31.832142,44957600,0.0,0.0,26.179647,NaN,...,26.206834,27.162465,25.865033,2,11,34.301074,14567.160156,2087.940918,16.194633,120.647763
3,2013-11-14,31.590239,31.807126,31.465114,31.715366,46183700,0.0,0.0,26.083608,NaN,...,26.255660,26.506778,26.116145,3,11,34.819073,14876.410156,2100.506104,16.194633,121.249714
4,2013-11-15,31.656982,31.715374,31.465122,31.565222,50601300,0.0,0.0,25.960125,NaN,...,26.053365,26.757903,26.011515,4,11,35.191377,15165.919922,2135.826904,16.194633,121.777288
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2512,2023-11-03,348.277763,353.019363,345.986641,351.435486,23624000,0.0,0.0,351.435486,66.777660,...,NaN,NaN,NaN,4,11,493.610881,32329.185547,3030.798096,27.280834,431.593750
2513,2023-11-06,352.083018,356.157196,351.983399,355.151093,23828300,0.0,0.0,355.151093,69.350009,...,NaN,NaN,NaN,0,11,492.420882,32708.480469,3058.410889,27.131378,432.586609
2514,2023-11-07,358.009986,361.058148,356.246842,359.135620,25833900,0.0,0.0,359.135620,73.299669,...,NaN,NaN,NaN,1,11,493.154729,32271.820312,3057.270020,27.041704,433.817749
2515,2023-11-08,360.281147,362.462680,359.155513,361.795288,26767800,0.0,0.0,361.795288,73.883761,...,NaN,NaN,NaN,2,11,503.569458,32166.480469,3052.373047,27.211088,434.135498


In [24]:
dataframes['MSFT'].columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends',
       'Stock Splits', 'Adjusted_Close', 'RSI', 'MACD', 'Signal_Line',
       'Upper_Band', 'Lower_Band', 'MA10', 'MA20', 'OBV', 'Target_1',
       'Target_2', 'Target_7', 'Target_14', 'Target_28', 'Day_of_Week',
       'Month', 'Adjusted_Close_MSCI', 'Adjusted_Close_^N225',
       'Adjusted_Close_000001.SS', 'Adjusted_Close_DAX', 'Adjusted_Close_SPY'],
      dtype='object')

## Phase 11: Exporting DataFrames to CSV

In this step, the refined stock data is saved into CSV files, with one file corresponding to each stock. Through the Build_DataFrames function, a CSV file is created for each entry in the stock data dictionary. The file names are composed of the stock's symbol and the specified date range, ensuring clarity and ease of identification.

Here’s what happens: A file name is generated for each stock, incorporating the stock's symbol and the date range of interest. A check is performed to see if a 'Dfs' folder, intended for storing these files, exists within the notebook's current location. If not found, this folder is automatically created. Subsequently, each stock's data is saved into a uniquely named CSV file within this foldeork.

In [25]:
def Build_DataFrames(dataframes, start_date, end_date):
    
    """
    Saves each DataFrame in the provided dictionary to a CSV file named after the ticker symbol.

    Parameters:
    - dataframes (dict): A dictionary where each key is a ticker symbol (string) and each value is a pandas DataFrame.
      The function iterates through this dictionary, saving each DataFrame to a CSV file.

    Note:
    This function does not return a value. Instead, it writes files to the disk, saving each DataFrame in the 'dataframes'
    dictionary to a separate CSV file.
    """
    
    for ticker in dataframes:
        # Concatenate the ticker symbol with '.csv' to form the filename        
        name_df = str(ticker+'_'+start_date+'_'+end_date+'.csv')
        
        # Get the current working directory (where your notebook is located)
        current_directory = os.getcwd()
        
        # Name of the subfolder where you want to save the CSV file
        subfolder_name = 'Dfs'

        # Construct the full path to the subfolder
        subfolder_path = os.path.join(current_directory, subfolder_name)
        
        # Check if the subfolder exists, if not, create it
        if not os.path.exists(subfolder_path):
            os.makedirs(subfolder_path)

        # Full path for the CSV file
        file_path = os.path.join(subfolder_path, name_df)

        # Save the DataFrame associated with the current ticker to a CSV file, excluding the index
        dataframes[ticker].to_csv(file_path, index=False)


In [26]:
Build_DataFrames(dataframes, start_date, end_date)